In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import seaborn as sns 
import matplotlib.colors

import time
from matplotlib.colors import LinearSegmentedColormap
pd.set_option('display.max_columns', None)


In [2]:
from nba_api.stats.static import teams

from nba_api.stats.endpoints import leaguedashplayerptshot, leaguedashplayerstats, synergyplaytypes

#### Get Touch Time DF

In [108]:
def get_touch_time_shots(season, touch_range):
    df = leaguedashplayerptshot.LeagueDashPlayerPtShot(season = season, touch_time_range_nullable = touch_range).get_data_frames()[0]
    
    
    df = df[['PLAYER_ID', 'PLAYER_LAST_TEAM_ID' , 'FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A']]
    
    shot_type = 'self_created'
    if touch_range == "Touch < 2 Seconds":
        shot_type = 'not_self_created'
        
    df['shot_type'] = shot_type
    return df

In [109]:
def get_touch_df(season):

    touch_range_list = ["Touch < 2 Seconds", "Touch 2-6 Seconds",  "Touch 6+ Seconds"]

    dataframes = []

    # Loop through touch_range_list and call the function for each value
    for touch_value in touch_range_list:
        print(touch_value)
        df = get_touch_time_shots(season = season, touch_range = touch_value)
        dataframes.append(df)
        time.sleep(3)
        
    touch_df = pd.concat(dataframes, ignore_index=True)

    touch_df = touch_df.groupby(['PLAYER_ID', 'PLAYER_LAST_TEAM_ID', 'shot_type']).agg(
        {**{col: 'first' for col in touch_df.columns if col not in ['PLAYER_ID', 'PLAYER_NAME', 'PLAYER_LAST_TEAM_ID', 'shot_type']},
        **{col: 'sum' for col in touch_df.columns if col not in ['PLAYER_ID', 'PLAYER_NAME', 'PLAYER_LAST_TEAM_ID', 'shot_type']}
        }
    ).reset_index()

    touch_df['efg'] = (touch_df.FGM + touch_df.FG3M*0.5)/touch_df.FGA

    suffix_mapping = {'not_self_created': '_not_self_created', 'self_created': '_self_created'}

    touch_df = touch_df.pivot_table(index=['PLAYER_ID', 'PLAYER_LAST_TEAM_ID' ], columns='shot_type', \
                                    values=['FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A', 'efg'], aggfunc='sum')

    # Flatten the MultiIndex columns
    touch_df.columns = [f'{col[0]}{suffix_mapping[col[1]]}' if col[1] in suffix_mapping else col[0] for col in touch_df.columns]

    touch_df = touch_df.reset_index()

    return(touch_df)

#### Get Synergy Data

In [110]:
def get_synergy_stats(season, play_type):

    synergy_df = synergyplaytypes.SynergyPlayTypes(season = season, play_type_nullable = play_type, per_mode_simple = 'Totals',\
                                      type_grouping_nullable = 'Offensive', player_or_team_abbreviation = 'P')\
                                .get_data_frames()[0]

    synergy_df = synergy_df[['PLAYER_ID', 'POSS', 'PTS']]


    synergy_df = synergy_df.groupby(['PLAYER_ID']).sum().reset_index()

    synergy_df['PPP'] = synergy_df.PTS/synergy_df.POSS

    for column in synergy_df.columns:
        # Check if the column name is not PLAYER_ID
        if column != 'PLAYER_ID':
            # Add the suffix based on the play_type variable
            new_column_name = column + "_" + play_type
            # Rename the column
            synergy_df.rename(columns={column: new_column_name}, inplace=True)

    return synergy_df

In [111]:
def get_synergy_df(season):

    synergy_df = poss_df[['PLAYER_ID']]

    play_types = ['Cut', 'Handoff', 'Isolation', 'OffScreen', 'Postup', 'PRBallHandler',\
                'PRRollman', 'Spotup', 'Transition']

    for play_type in play_types:
        print(play_type)
        synergy_df = pd.merge(synergy_df, get_synergy_stats(season, play_type),  how = 'left')
        time.sleep(3)

    return(synergy_df)
    

### Pull Data & Cleanse

In [128]:
season = '2023-24'

##### Pull Possession Data

In [129]:
poss_df = leaguedashplayerstats.LeagueDashPlayerStats(season=season, \
                                                  measure_type_detailed_defense='Advanced').get_data_frames()[0]



poss_df = poss_df[['PLAYER_ID', 'PLAYER_NAME', 'POSS', 'GP', 'MIN']]

poss_df = poss_df.groupby(['PLAYER_ID', 'PLAYER_NAME']).sum().reset_index()

##### Pull Stats Data

In [130]:
print('\nPulling Touch Time Data')
touch_df = get_touch_df(season)

print('\nPulling Synergy Data')
synergy_df = get_synergy_df(season)


Pulling Touch Time Data
Touch < 2 Seconds
Touch 2-6 Seconds
Touch 6+ Seconds

Pulling Synergy Data
Cut
Handoff
Isolation
OffScreen
Postup
PRBallHandler
PRRollman
Spotup
Transition


##### Data Cleanse & Merge

In [131]:
off_ball_df = pd.merge(poss_df, touch_df, on = 'PLAYER_ID')

off_ball_df = pd.merge(off_ball_df, synergy_df, on = 'PLAYER_ID')


for column in off_ball_df.columns:
    if (column.startswith("FG") or column.startswith("PTS") or column.startswith("POSS_")):
        off_ball_df[column] = (off_ball_df[column] / off_ball_df['POSS']) * 75
        
#off_ball_df = off_ball_df[off_ball_df.POSS > 1000]

off_ball_df['headshot_url'] = "https://cdn.nba.com/headshots/nba/latest/1040x760/" + \
            off_ball_df.PLAYER_ID.astype(str) + \
                    ".png"

In [132]:
team_colors = pd.read_csv('Data/teamColors.csv')
team_colors.rename(columns={'TEAM_ID': 'PLAYER_LAST_TEAM_ID', 'Primary Color': 'primary_color'}, inplace=True)
team_colors = team_colors[['PLAYER_LAST_TEAM_ID', 'primary_color']]

off_ball_df = pd.merge(off_ball_df, team_colors, on = 'PLAYER_LAST_TEAM_ID')

##### Export Data

In [133]:
off_ball_df['season'] = season

off_ball_df.to_csv('Data/headshot_plot_obs_' + season + '.csv')

### Save Latest Update Time

In [134]:
from datetime import date

# Get today's date
today = date.today()

# Convert date to string
today_str = today.strftime("%Y-%m-%d")

# Define the file name
file_name = "latest_update.txt"

# Save the date as a text file
with open(file_name, "w") as file:
    file.write(today_str)

print(f"Today's date ({today_str}) saved in {file_name}")

Today's date (2024-12-17) saved in latest_update.txt


In [135]:
off_ball_df[off_ball_df.PLAYER_LAST_TEAM_ID == 1610612751].sort_values(by = 'FGA_self_created', ascending = False)


,PLAYER_ID,PLAYER_NAME,POSS,GP,MIN,PLAYER_LAST_TEAM_ID,FG2A_not_self_created,FG2A_self_created,FG2M_not_self_created,FG2M_self_created,FG3A_not_self_created,FG3A_self_created,FG3M_not_self_created,FG3M_self_created,FGA_not_self_created,FGA_self_created,FGM_not_self_created,FGM_self_created,efg_not_self_created,efg_self_created,POSS_Cut,PTS_Cut,PPP_Cut,POSS_Handoff,PTS_Handoff,PPP_Handoff,POSS_Isolation,PTS_Isolation,PPP_Isolation,POSS_OffScreen,PTS_OffScreen,PPP_OffScreen,POSS_Postup,PTS_Postup,PPP_Postup,POSS_PRBallHandler,PTS_PRBallHandler,PPP_PRBallHandler,POSS_PRRollman,PTS_PRRollman,PPP_PRRollman,POSS_Spotup,PTS_Spotup,PPP_Spotup,POSS_Transition,PTS_Transition,PPP_Transition,headshot_url,primary_color,season
381,1630560,Cam Thomas,4321,66,31.4,1610612751,1.440639,11.750752,0.798426,5.554270,3.419347,3.054848,1.423282,0.919926,4.859986,14.805601,2.221708,6.474196,0.603571,0.468347,0.329785,0.312428,0.947368,2.898635,2.811849,0.970060,3.662347,3.488776,0.952607,0.555427,0.555427,1.000000,NaN,NaN,NaN,7.671835,6.942837,0.904977,0.190928,0.260356,1.363636,4.912057,5.831983,1.187279,NaN,NaN,NaN,https://cdn.nba.com/headshots/nba/latest/1040x...,#000000,2023-24
217,1629022,Lonnie Walker IV,2119,58,17.4,1610612751,1.274186,6.229353,0.637093,2.831524,6.512506,2.866918,2.583766,1.097216,7.786692,9.096272,3.220859,3.928740,0.579545,0.492218,NaN,NaN,NaN,1.380368,1.274186,0.923077,1.238792,1.132610,0.914286,0.778669,0.566305,0.727273,NaN,NaN,NaN,4.884379,4.070316,0.833333,NaN,NaN,NaN,6.831052,7.149599,1.046632,NaN,NaN,NaN,https://cdn.nba.com/headshots/nba/latest/1040x...,#000000,2023-24
63,203471,Dennis Schröder,5106,80,31.1,1610612751,0.940071,7.197415,0.484724,3.363690,4.259694,1.042891,1.659812,0.337838,5.199765,8.240306,2.144536,3.701528,0.572034,0.469697,0.190952,0.190952,1.000000,1.233843,1.263220,1.023810,1.703878,1.366040,0.801724,0.205640,0.220329,1.071429,NaN,NaN,NaN,6.565805,5.713866,0.870246,NaN,NaN,NaN,4.200940,4.612221,1.097902,1.777321,1.659812,0.933884,https://cdn.nba.com/headshots/nba/latest/1040x...,#000000,2023-24
184,1628969,Mikal Bridges,5852,82,34.8,1610612751,1.819891,6.895079,1.127820,3.152768,6.010766,1.332878,2.306904,0.435748,7.830656,8.227956,3.434723,3.588517,0.585925,0.462617,0.794600,1.050923,1.322581,2.050581,2.101846,1.025000,1.525120,1.102187,0.722689,1.102187,1.230349,1.116279,0.269139,0.320403,1.190476,4.806049,4.088346,0.850667,0.179426,0.179426,1.000000,NaN,NaN,NaN,3.319378,3.806391,1.146718,https://cdn.nba.com/headshots/nba/latest/1040x...,#000000,2023-24
388,1630570,Trendon Watford,1761,63,13.6,1610612751,2.683135,7.027257,1.618399,4.003407,2.512777,0.212947,1.022147,0.042589,5.195911,7.240204,2.640545,4.045997,0.606557,0.561765,1.235094,1.448041,1.172414,1.149915,1.490630,1.296296,0.894378,1.149915,1.285714,NaN,NaN,NaN,0.681431,0.638842,0.937500,NaN,NaN,NaN,0.553663,0.553663,1.000000,4.344123,4.216354,0.970588,2.853492,3.364566,1.179104,https://cdn.nba.com/headshots/nba/latest/1040x...,#000000,2023-24
153,1628372,Dennis Smith Jr.,2160,56,18.9,1610612751,1.493056,6.319444,1.111111,2.812500,2.812500,0.729167,0.868056,0.173611,4.305556,7.048611,1.979167,2.986111,0.560484,0.435961,0.555556,0.833333,1.500000,0.555556,0.763889,1.375000,1.527778,0.868056,0.568182,NaN,NaN,NaN,NaN,NaN,NaN,3.090278,2.361111,0.764045,NaN,NaN,NaN,4.270833,3.541667,0.829268,NaN,NaN,NaN,https://cdn.nba.com/headshots/nba/latest/1040x...,#000000,2023-24
375,1630553,Keon Johnson,128,5,12.3,1610612751,2.343750,3.515625,1.171875,0.585938,3.515625,1.757812,2.343750,0.000000,5.859375,5.273438,3.515625,0.585938,0.800000,0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.nba.com/headshots/nba/latest/1040x...,#000000,2023-24
269,1629661,Cameron Johnson,3297,58,27.6,1610612751,1.819836,4.140127,1.023658,2.047316,6.869882,1.023658,2.866242,0.204732,8.689718,5.163785,3.889900,2.252047,0.612565,0.455947,0.614195,0.750682,1.222222,1.296633,1.091902,0.842105,0.5686

In [137]:
# Filter the DataFrame
filtered_df = off_ball_df[off_ball_df.POSS > 1500].copy()

# Calculate percentiles for 'efg_self_created'
filtered_df['efg_self_created_percentile'] = filtered_df['efg_self_created'].rank(pct=True) * 100


filtered_df[filtered_df.PLAYER_NAME == 'RJ Barrett']


,PLAYER_ID,PLAYER_NAME,POSS,GP,MIN,PLAYER_LAST_TEAM_ID,FG2A_not_self_created,FG2A_self_created,FG2M_not_self_created,FG2M_self_created,FG3A_not_self_created,FG3A_self_created,FG3M_not_self_created,FG3M_self_created,FGA_not_self_created,FGA_self_created,FGM_not_self_created,FGM_self_created,efg_not_self_created,efg_self_created,POSS_Cut,PTS_Cut,PPP_Cut,POSS_Handoff,PTS_Handoff,PPP_Handoff,POSS_Isolation,PTS_Isolation,PPP_Isolation,POSS_OffScreen,PTS_OffScreen,PPP_OffScreen,POSS_Postup,PTS_Postup,PPP_Postup,POSS_PRBallHandler,PTS_PRBallHandler,PPP_PRBallHandler,POSS_PRRollman,PTS_PRRollman,PPP_PRRollman,POSS_Spotup,PTS_Spotup,PPP_Spotup,POSS_Transition,PTS_Transition,PPP_Transition,headshot_url,primary_color,season,efg_self_created_percentile
247,1629628,RJ Barrett,3889,58,31.7,1610612761,3.394189,8.678323,2.159938,4.454873,4.570584,0.154281,1.697094,0.019285,7.964772,8.832605,3.857033,4.474158,0.590799,0.507642,NaN,NaN,NaN,1.79352,1.658524,0.924731,0.752121,0.63641,0.846154,0.443559,0.520699,1.173913,0.713551,0.655696,0.918919,3.085626,2.372075,0.76875,NaN,NaN,NaN,2.622782,2.410645,0.919118,5.881975,6.904088,1.17377,https://cdn.nba.com/headshots/nba/latest/1040x...,#CE1141,2023-24,59.68254
